In [51]:
# # Building Name & State Dimensions & Control Dimensions & Num. of Constraints & Main Objective (Minimize) \\
# from energym.wrappers.rl_wrapper import StableBaselinesRLWrapper
# from buildings_factory import *
# objectives = ["average exchanged energy",
#               "average exchanged energy",
#               "average exchanged energy",
#               "average exchanged energy",
#               "power demand",
#               "power demand",
#               "CO2 emission",
#               "CO2 emission",
#               "power demand",
#               "power demand",
#               "power demand",
#               "power demand"]

# model_info = ""
# for building_name in buildings_list:
#     # min_kpis, max_kpis = collect_baseline_kpi(building_name)
#     building_idx = buildings_list.index(building_name)
#     env = get_env(building_name)
#     inputs = get_inputs(building_name, env)
#     default_control = default_controls[building_idx]
#     env_rl = StableBaselinesRLWrapper(building_name, None, None, reward_func)
#     input_dim = env_rl.observation_space.shape[0]
#     control_dim = env_rl.action_space.shape[0]
#     constraint_num = 0
#     for key, val in env.kpi_options.items():
#         if val["type"] == 'avg_dev': constraint_num += 1 
#     main_objective = objectives[building_idx]
#     model_info += f"{building_name} & {input_dim} & {control_dim} & {constraint_num} & {main_objective} \\ "
# print(model_info)

In [52]:
import pandas as pd
# buildings_list = ["ApartmentsThermal-v0", "ApartmentsGrid-v0", "Apartments2Thermal-v0",
#                   "Apartments2Grid-v0", "OfficesThermostat-v0", "MixedUseFanFCU-v0",
#                   "SeminarcenterThermostat-v0", "SeminarcenterFull-v0", "SimpleHouseRad-v0",
#                   "SimpleHouseRSla-v0", "SwissHouseRSlaW2W-v0", "SwissHouseRSlaTank-v0"] 

# plot_building_names = ["SimpleHouseRad-v0", "SimpleHouseRSla-v0", 
#                         "SwissHouseRSlaW2W-v0", "SwissHouseRSlaTank-v0"]

plot_building_names = ["OfficesThermostat-v0", "MixedUseFanFCU-v0", "SimpleHouseRad-v0", "SimpleHouseRSla-v0", 
                        "SwissHouseRSlaW2W-v0", "SwissHouseRSlaTank-v0"]
seeds = [7, 13, 17, 19]

# "MOPO_ori_simulator"
model_names = ["TD3PlusBC_ori_simulator{}", "CQL_ori_simulator{}", "MOPO_ori_simulator{}",
               "SAC_1_dnn_predictor_alpha2{}", "SAC_4_dnn_predictor_alpha2{}", "SAC_8_dnn_predictor_alpha2{}"]
legend_names = ["TD3PlusBC", "CQL", "MOPO", "Ours_1", "Ours_4", "Ours_8"]

len_traj = 1

# model_names = ["SAC_{}_dnn_predictor{}_alpha1", "SAC_{}_dnn_predictor{}_alpha2",
#                "SAC_{}_dnn_predictor{}_alpha4", "SAC_{}_dnn_predictor{}_alpha6",
#                "SAC_{}_dnn_predictor{}_alpha8", "SAC_{}_dnn_predictor{}_alpha10"]
# legend_names = ["alpha1" , "alpha2", "alpha4", "alpha6", "alpha8", "alpha10"] 

# model_names = ["TD3PlusBC_ori_simulator{}", "CQL_ori_simulator{}", "MOPO_ori_simulator{}",
#                "SAC_1_dnn_predictor_alpha1{}", "SAC_1_dnn_predictor_alpha2{}",
#                "SAC_1_dnn_predictor_alpha4{}", "SAC_1_dnn_predictor_alpha6{}",
#                "SAC_1_dnn_predictor_alpha8{}", "SAC_1_dnn_predictor_alpha10{}"]
# legend_names = ["TD3PlusBC", "CQL", "MOPO", "alpha1" , "alpha2", "alpha4", "alpha6", "alpha8", "alpha10"] 

# model_names = ["TD3PlusBC_ori_simulator", "CQL_ori_simulator", "SAC_bs_predictor", "SAC_dnn_predictor"]
# legend_names = ["TD3PlusBC", "CQL", "Best", "Ours"]
res_cols = ["manual_baseline_eval_episode_reward", "manual_eval_episode_reward"]
            
colors = ['r', 'g', 'r--', 'g--']

res_data = []
for seed in seeds:
    for _model_name, legend_name in zip(model_names, legend_names):
        for building in plot_building_names:
            alpha_col = "" # ("" if not legend_name.startswith("Ours") else "_alpha2")
            model_name = _model_name.format(f"{alpha_col}_seed{seed}")
            res_loc = f"data/data_aml/models/{building}/{model_name}/logs/best/rewards.csv"
            res_row = pd.read_csv(res_loc).iloc[-1, :]
            model_col_name = _model_name.format("")
            res_data.append([seed, building, model_col_name, legend_name,
                             res_row["eval_episode_reward"], res_row["manual_eval_episode_reward"],
                             res_row["baseline_eval_episode_reward"], res_row["manual_baseline_eval_episode_reward"]])
res_df = pd.DataFrame(res_data, columns=["seed", "building", "model_name", 
                                         "legend", "learnt_reward", "manual_reward",
                                         "learnt_baseline_reward", "manual_baseline_reward"])
            

res_df.head()

,seed,building,model_name,legend,learnt_reward,manual_reward,learnt_baseline_reward,manual_baseline_reward
0,7,OfficesThermostat-v0,TD3PlusBC_ori_simulator,TD3PlusBC,-417980.141387,-417980.141387,-221801.892252,-221801.892252
1,7,MixedUseFanFCU-v0,TD3PlusBC_ori_simulator,TD3PlusBC,-52384.744473,-52384.744473,-61677.972402,-61677.972402
2,7,SimpleHouseRad-v0,TD3PlusBC_ori_simulator,TD3PlusBC,-17552.595889,-17552.595889,-11657.517872,-11657.517872
3,7,SimpleHouseRSla-v0,TD3PlusBC_ori_simulator,TD3PlusBC,-17471.306517,-17471.306517,-12827.544447,-12827.544447
4,7,SwissHouseRSlaW2W-v0,TD3PlusBC_ori_simulator,TD3PlusBC,-37477.534607,-37477.534607,-18575.031798,-18575.031798


In [53]:
import numpy as np
cols = res_df.columns.tolist()
# "RL_reward", "manual_RL_reward", "baseline_reward", "manual_baseline_reward"
cols.remove("seed")
stat_df = res_df[cols].groupby(["building", "model_name", "legend"]).agg(learnt_reward_mean=("learnt_reward", np.mean),
                                                           learnt_reward_std=("learnt_reward", np.std),
                                                           learnt_baseline_reward_mean=("learnt_baseline_reward", np.mean),
                                                           learnt_baseline_reward_std=("learnt_baseline_reward", np.std),
                                                           manual_reward_mean=("manual_reward", np.mean),
                                                           manual_reward_std=("manual_reward", np.std),
                                                           manual_baseline_reward_mean=("manual_baseline_reward", np.mean),
                                                           manual_baseline_reward_std=("manual_baseline_reward", np.std)).reset_index()

stat_df.fillna(0.0, inplace=True)


In [54]:
bs_stat_df = stat_df[['building', "manual_baseline_reward_mean"]].groupby("building").mean().reset_index()
bs_stat_df.columns = ['building', 'manual_reward_mean']
bs_stat_df['legend'] = 'baseline'
bs_stat_df['manual_reward_std'] = 0.0
bs_stat_df.head()

,building,manual_reward_mean,legend,manual_reward_std
0,MixedUseFanFCU-v0,-26714.744817,baseline,0.0
1,OfficesThermostat-v0,-90670.803165,baseline,0.0
2,SimpleHouseRSla-v0,-13148.024822,baseline,0.0
3,SimpleHouseRad-v0,-11684.173973,baseline,0.0
4,SwissHouseRSlaTank-v0,-21033.790452,baseline,0.0


In [58]:
cols = ["building", "legend", "manual_reward_mean", "manual_reward_std"]
stat_df_all = pd.concat([stat_df[cols], bs_stat_df[cols]])
stat_df_all['reward'] = stat_df_all.apply(lambda x: (f"{x['manual_reward_mean']:.0f}\xb1{x['manual_reward_std']:.0f}" if x['legend'] != "baseline" else f"{x['manual_reward_mean']:.0f}"), axis=1)
df = stat_df_all.pivot(index='building', columns='legend', values='reward').reset_index()
df.sort_values(by="building", inplace=True)
df

legend,building,CQL,MOPO,Ours_1,Ours_4,Ours_8,TD3PlusBC,baseline
0,MixedUseFanFCU-v0,-33036±7777,-9284±1928,-4899±860,-3977±361,-5255±1551,-52284±2172,-26715
1,OfficesThermostat-v0,-172464±43809,-24718±6784,-17684±7142,-16012±3435,-20561±9554,-476753±77803,-90671
2,SimpleHouseRSla-v0,-19120±18826,-33941±11053,-6868±667,-6393±530,-5930±448,-17721±1728,-13148
3,SimpleHouseRad-v0,-36359±46588,-9104±4080,-5995±453,-6727±1122,-6054±194,-21740±7078,-11684
4,SwissHouseRSlaTank-v0,-8849±2565,-11587±382,-4716±424,-5081±1070,-5923±1176,-8634±2547,-21034
5,SwissHouseRSlaW2W-v0,-26248±24786,-39313±22731,-4557±106,-8232±1068,-6860±698,-19828±11952,-20678


In [59]:
# print_cols = ["building", "baseline", "CQL", "TD3PlusBC", "MOPO", "alpha1", "alpha2", "alpha4", "alpha6", "alpha8", "alpha10"]
print_cols = ["building", "baseline", "CQL", "TD3PlusBC", "MOPO", "Ours"] # "Ours_1", "Ours_4", "Ours_8"]
# print_cols = ["building", "baseline", "Ours_1", "Ours_4", "Ours_8"]
print(" & ".join(print_cols), "\\\\")
print("\\hline")
for _, row in df.iterrows():
    print(" & ".join([str(row[col]) for col in print_cols]), "\\\\")
    print("\\hline")

TypeError: unsupported operand type(s) for /: 'str' and 'int'